<a href="https://colab.research.google.com/github/Alish7495/Retrieval-Augmented-Generation/blob/main/test_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*# Imagine a company that has hundreds of internal regulations — HR rules, employee benefits, leave policies, dress code, and so on.*
--------------------------------------------------------------------------------
*# Employees often need to look up these rules quickly, but searching long PDF documents or files can be time-consuming.*
--------------------------------------------------------------------------------
# To solve that, we can build a Retrieval-Augmented Generation (RAG) system that connects a language model to the company’s internal regulation database.
# The model can retrieve the most relevant policy sections and generate clear, conversational answers for employees — just like an intelligent HR assistant. *

## **[Step1: Install the libraries]**

In [ ]:
# @title
!pip install chromadb langchain langchain-openai langchain-community langserve fastapi uvicorn langsmith openai tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.5 MB/s eta 0:00:00
   ━

## [**Step 2: Mounting google drive in case if your data is stored there**]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
BASE_DIR = "/content/drive/MyDrive/company_rag"
os.makedirs(BASE_DIR, exist_ok=True)


Mounted at /content/drive


## **Step 3 : Access and import your API keys for Open AI and LangSmith: We are using openAI models for embedding and generation and the Use Langsmith for tracing our retrieval and generation**

In [ ]:
import os, getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# (Optional) LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_PROJECT"] = "company-regulations-rag"


Enter your OpenAI API key: ··········
Enter your LangSmith API key: ··········


## **Step 4 : We create a sets of instruction for the company: The Regulations!**

In [ ]:
DATA_PATH = f"{BASE_DIR}/company_regulations.jsonl"

jsonl_text = """{"id":"rule_001","topic":"Working Hours","content":"Standard working hours are 09:00–18:00 Monday through Friday with a 1-hour unpaid lunch break. Employees must record arrival and departure times in the company timekeeping system."}
{"id":"rule_002","topic":"Attendance & Punctuality","content":"Employees should be present and ready to work at their scheduled start time. Repeated lateness without prior notice may lead to disciplinary action."}
{"id":"rule_003","topic":"Remote Work Eligibility","content":"Remote work is allowed only with manager approval. Eligibility is based on job function, performance, and business needs; remote schedules must be agreed in writing."}
{"id":"rule_004","topic":"Overtime Authorization","content":"Overtime must be approved in advance by a manager. Non-exempt employees will be compensated according to local wage laws for approved overtime hours."}
{"id":"rule_005","topic":"Paid Time Off (PTO) Accrual","content":"PTO accrues monthly per the company accrual schedule. Employees must request PTO through the HR portal and receive manager approval before booking travel."}
{"id":"rule_006","topic":"Sick Leave","content":"Employees should notify their manager as soon as possible when sick. Sick leave usage should be recorded in the HR system; documentation may be requested for extended absences."}
{"id":"rule_007","topic":"Parental Leave","content":"Eligible employees may take parental leave consistent with local law and company policy. Requests should be submitted to HR at least 30 days in advance when practicable."}
{"id":"rule_008","topic":"Bereavement Leave","content":"Employees may take bereavement leave for the death of immediate family members. The duration and pay will follow the company bereavement policy and local regulations."}
{"id":"rule_009","topic":"Dress Code","content":"Employees should dress in a manner appropriate for their role and customers. Casual dress is allowed unless client meetings or formal events require business attire."}
{"id":"rule_010","topic":"Workplace Conduct","content":"All employees must treat colleagues, customers, and vendors with respect. Harassment, bullying, or discriminatory behavior is strictly prohibited and will be investigated."}
{"id":"rule_011","topic":"Equal Opportunity","content":"The company is an equal opportunity employer. Hiring, promotion, and compensation decisions are based on merit and without discrimination."}
{"id":"rule_012","topic":"Anti-Harassment","content":"Harassment of any kind is prohibited. Employees should report incidents to HR or via the anonymous reporting channel; the company will respond promptly and confidentially."}
{"id":"rule_013","topic":"Conflict of Interest","content":"Employees must disclose any personal, financial, or external interests that could conflict with company duties. Undisclosed conflicts may result in corrective action."}
{"id":"rule_014","topic":"Gifts & Entertainment","content":"Employees may not accept gifts or entertainment that could influence business decisions. All offers above the prescribed monetary limit must be reported and approved."}
{"id":"rule_015","topic":"Bribery & Corruption","content":"The company prohibits bribery or corrupt practices. Employees must comply with anti-bribery laws and report suspicious requests or offers immediately."}
{"id":"rule_016","topic":"Confidentiality","content":"Employees must protect confidential company information and customer data. Sharing confidential information outside approved channels is forbidden unless authorized."}
{"id":"rule_017","topic":"Non-Disclosure Agreements (NDA)","content":"Employees are required to sign NDAs when handling sensitive information. NDAs survive employment and prohibit unauthorized disclosure after separation."}
{"id":"rule_018","topic":"Intellectual Property","content":"Work created during employment belongs to the company unless a written agreement states otherwise. Employees should promptly disclose inventions or innovations to legal."}
{"id":"rule_019","topic":"Data Protection & Privacy","content":"Personal data must be processed lawfully, fairly, and transparently. Employees must follow data handling procedures and report breaches to the Data Protection Officer."}
{"id":"rule_020","topic":"Password & Access Management","content":"Employees must use strong passwords, enable multi-factor authentication where available, and not share credentials. Passwords must be rotated per IT policy."}
{"id":"rule_021","topic":"Device Use & BYOD","content":"Personal devices used for work must comply with security requirements and be registered with IT. Sensitive data should not be stored on unmanaged devices."}
{"id":"rule_022","topic":"Email Use","content":"Company email is for business use. Employees should avoid excessive personal use and not send sensitive data unencrypted. Phishing attempts must be reported immediately."}
{"id":"rule_023","topic":"Internet & Acceptable Use","content":"Company networks must be used responsibly. Accessing illegal or inappropriate content is prohibited; bandwidth-intensive personal usage should be minimized."}
{"id":"rule_024","topic":"Software Installation","content":"Only IT-approved software may be installed on company devices. Unauthorized software can create security and compliance risks and will be removed."}
{"id":"rule_025","topic":"Open Source & Code Repositories","content":"Contributions to open-source projects must be reviewed for IP and licensing compliance. Company code must reside in approved repositories with proper access controls."}
{"id":"rule_026","topic":"Remote Access & VPN","content":"Remote access to internal systems requires the company VPN and MFA. Access rights are granted by role and reviewed periodically by IT."}
{"id":"rule_027","topic":"Two-Factor Authentication (2FA)","content":"Critical systems require two-factor authentication. Employees must set up 2FA on accounts that support it and never circumvent security controls."}
{"id":"rule_028","topic":"Data Backup & Retention","content":"Important business data must be backed up per IT schedules. Records retention periods are defined by department and legal requirements; do not delete retained records."}
{"id":"rule_029","topic":"Information Classification","content":"Employees must classify information as Public, Internal, Confidential, or Restricted and handle it according to the corresponding safeguards."}
{"id":"rule_030","topic":"Encryption","content":"Sensitive data must be encrypted in transit and at rest according to IT standards. Unencrypted transfer of sensitive files is not permitted."}
{"id":"rule_031","topic":"Physical Security & Badges","content":"Employees must display company badges while on premises and report lost badges to security immediately. Tailgating through secured doors is prohibited."}
{"id":"rule_032","topic":"Visitor Policy","content":"All visitors must be registered and escorted while on company premises. Visitors must follow safety and confidentiality rules and wear visitor badges."}
{"id":"rule_033","topic":"Workplace Safety","content":"Employees must follow safety procedures and report hazards. Participation in required safety training and drills is mandatory."}
{"id":"rule_034","topic":"Emergency Evacuation","content":"Follow posted evacuation routes and assembly points during alarms. Do not re-enter buildings until declared safe by authorities."}
{"id":"rule_035","topic":"Fire Safety","content":"Keep exits clear and do not obstruct fire extinguishers. Report electrical faults immediately; tampering with safety equipment is prohibited."}
{"id":"rule_036","topic":"Drug & Alcohol-Free Workplace","content":"The use or possession of illegal drugs or intoxication at work is forbidden. Prescription medications should be used responsibly and disclosed to HR if they impair performance."}
{"id":"rule_037","topic":"Workplace Violence","content":"Threats or acts of violence are prohibited. Report any violent behavior or credible threats to security or HR immediately."}
{"id":"rule_038","topic":"Smoking Policy","content":"Smoking is allowed only in designated outdoor areas. Smoking inside company buildings is prohibited and subject to fines per local law."}
{"id":"rule_039","topic":"Parking & Vehicle Use","content":"Parking permits are required for company lots. Use of company vehicles requires a valid license and adherence to the vehicle use policy."}
{"id":"rule_040","topic":"Business Travel & Expense Reimbursement","content":"Business travel must be pre-approved. Submit expenses with receipts within 30 days; reimbursement follows company per-diem and approval rules."}
{"id":"rule_041","topic":"Corporate Credit Card Use","content":"Company cards are for approved business expenses only. Cardholders must retain receipts and reconcile statements monthly; misuse may result in revocation."}
{"id":"rule_042","topic":"Procurement & Vendor Selection","content":"Purchases above defined thresholds require competitive quotes and approvals. Vendor selection must consider cost, quality, compliance, and conflict-of-interest disclosures."}
{"id":"rule_043","topic":"Gift & Vendor Hospitality","content":"Accepting gifts or hospitality from vendors is restricted. Report offers above the de minimis threshold and obtain manager approval when necessary."}
{"id":"rule_044","topic":"Contract Approval","content":"Contracts obligating the company must be reviewed by Legal and approved according to delegation limits. Verbal commitments are not binding."}
{"id":"rule_045","topic":"Anti-Money Laundering (AML)","content":"Employees must follow AML due diligence when onboarding customers or processing transactions. Suspicious activity must be reported to Compliance."}
{"id":"rule_046","topic":"Export Controls & Trade Compliance","content":"Export-controlled goods, software, and information require screening and licenses as applicable. Export violations may result in severe penalties."}
{"id":"rule_047","topic":"Insider Trading","content":"Employees with material non-public information must not trade in company securities or tip others; seek Legal guidance when in doubt."}
{"id":"rule_048","topic":"Whistleblower & Reporting","content":"Employees may report misconduct confidentially or anonymously. Retaliation against reporters is strictly prohibited and will be disciplined."}
{"id":"rule_049","topic":"Performance Reviews","content":"Performance reviews occur at least annually. Objectives and development plans should be documented and discussed with employees regularly."}
{"id":"rule_050","topic":"Probationary Period","content":"New hires serve a probationary period during which performance and fit are evaluated. Managers should provide timely feedback and support."}
{"id":"rule_051","topic":"Promotions & Internal Mobility","content":"Open roles are posted internally when appropriate. Promotions are based on performance, skills, and business needs, not tenure alone."}
{"id":"rule_052","topic":"Compensation & Salary Reviews","content":"Compensation decisions adhere to pay bands and market data. Salary reviews occur per the company cycle and depend on performance and budget."}
{"id":"rule_053","topic":"Benefits Enrollment","content":"Employees must enroll in benefits within the enrollment window. Changes outside the window require qualifying life events and supporting documentation."}
{"id":"rule_054","topic":"Retirement Plan","content":"Eligible employees may participate in the company retirement plan according to plan terms. Contributions and vesting schedules are described in plan documents."}
{"id":"rule_055","topic":"Health & Wellness Programs","content":"The company offers wellness initiatives and encourages participation. Medical information shared for wellness is handled confidentially."}
{"id":"rule_056","topic":"Training & Development","content":"Employees are expected to complete mandatory training and pursue development aligned with role requirements. Managers should allocate time for learning."}
{"id":"rule_057","topic":"Tuition Assistance","content":"Tuition assistance may be available for approved programs. Applications require manager endorsement and are subject to budget and return-on-investment criteria."}
{"id":"rule_058","topic":"Intern & Contractor Management","content":"Interns and contractors must have onboarding paperwork and defined supervisors. Contractors follow company policies while on site or using company systems."}
{"id":"rule_059","topic":"Background Checks","content":"Employment offers may be contingent on successful background checks per local law. Applicants must consent to checks required for the role."}
{"id":"rule_060","topic":"Hiring & Reference Checks","content":"Hiring managers must conduct reference checks for final candidates. Offer letters must be approved by HR and include required terms and conditions."}
{"id":"rule_061","topic":"Recruitment Referral Program","content":"Employee referrals are encouraged and may be eligible for referral bonuses if the hire passes probation. Referral rules are posted on the intranet."}
{"id":"rule_062","topic":"Resignation & Notice Period","content":"Employees should provide advance written notice according to their contract (typically 2–4 weeks). Managers should plan knowledge transfer and handover tasks."}
{"id":"rule_063","topic":"Exit Interviews","content":"HR conducts exit interviews to capture feedback. Departing employees must return company property and complete offboarding procedures."}
{"id":"rule_064","topic":"Asset Management & Tagging","content":"Company assets must be tagged and tracked. Employees are responsible for the safekeeping of assigned devices and must report loss or damage."}
{"id":"rule_065","topic":"Asset Disposal & Data Wiping","content":"Disposed devices must be wiped per IT standards and handled through approved disposal channels to prevent data leakage."}
{"id":"rule_066","topic":"Facility Cleanliness & Housekeeping","content":"Employees are responsible for keeping workspaces tidy. Common areas should be left clean after use to maintain a professional environment."}
{"id":"rule_067","topic":"Meeting Etiquette","content":"Start and end meetings on time, share agendas in advance, and respect attendees’ time. Use video/audio norms agreed by the team for remote meetings."}
{"id":"rule_068","topic":"Video Conferencing & Recording","content":"Recordings require participant consent and must be stored securely with access limited to authorized personnel."}
{"id":"rule_069","topic":"Meeting Notes & Action Items","content":"Assign owners and deadlines for action items during meetings. Post concise notes to the team workspace within 24 hours."}
{"id":"rule_070","topic":"Social Media Use","content":"Employees must distinguish personal views from company position. Posting confidential information or disparaging content about the company is prohibited."}
{"id":"rule_071","topic":"Media Inquiries","content":"Only authorized spokespeople may respond to media requests. Refer journalists to Corporate Communications for official statements."}
{"id":"rule_072","topic":"Brand & Logo Use","content":"Use of company logos and brand assets requires approval and adherence to brand guidelines for color, layout, and messaging."}
{"id":"rule_073","topic":"Corporate Donations & Sponsorships","content":"Charitable donations and sponsorships require approval and alignment with company charitable giving guidelines and budget controls."}
{"id":"rule_074","topic":"Political Activity","content":"Employees may participate in political activities on their own time but should not use company resources or imply company endorsement."}
{"id":"rule_075","topic":"Sustainability & Energy Use","content":"Employees should conserve energy, follow recycling programs, and support company sustainability initiatives to reduce environmental impact."}
{"id":"rule_076","topic":"Health & Ergonomics","content":"Workstations should be ergonomically set up. Report discomfort to HR or Occupational Health so adjustments or equipment can be provided."}
{"id":"rule_077","topic":"Occupational Health Reporting","content":"Report work-related injuries or illnesses to HR immediately. The company will assist with medical care and necessary reporting to authorities."}
{"id":"rule_078","topic":"Workplace Accommodations","content":"The company provides reasonable accommodations for disabilities. Employees should contact HR to request accommodations and provide supporting documentation."}
{"id":"rule_079","topic":"Childcare & Family Support","content":"Where available, the company may offer family support programs. Eligibility, application processes, and subsidies are described in benefits materials."}
{"id":"rule_080","topic":"Internal Communication Channels","content":"Use designated communication channels for work. Sensitive discussions should occur in secure spaces and not on public or unapproved platforms."}
{"id":"rule_081","topic":"Document Management","content":"Store official documents in approved repositories. Follow naming conventions and version control to ensure discoverability and compliance."}
{"id":"rule_082","topic":"Record Retention & Legal Holds","content":"Do not delete documents subject to a legal hold. Legal will notify relevant employees when a hold is in place and provide instructions."}
{"id":"rule_083","topic":"Quality Assurance","content":"Products and services must meet defined quality standards. Report defects and participate in corrective actions to ensure customer satisfaction."}
{"id":"rule_084","topic":"Customer Data Handling","content":"Handle customer data in line with contracts and privacy policies. Only access customer information necessary to perform job duties."}
{"id":"rule_085","topic":"Customer Complaints & Escalation","content":"Log customer complaints promptly and follow escalation procedures. Managers must resolve issues within defined SLAs and communicate updates."}
{"id":"rule_086","topic":"Service Level Agreements (SLA) Compliance","content":"Teams must meet SLA commitments. Monitor performance metrics and escalate risks to leadership when targets are at risk."}
{"id":"rule_087","topic":"Product Safety & Compliance","content":"Ensure products comply with applicable safety standards and regulations. Non-compliance must be reported to Product and Legal immediately."}
{"id":"rule_088","topic":"Change Management","content":"Significant system or process changes require documented change requests, testing, and approvals before production rollout to avoid disruptions."}
{"id":"rule_089","topic":"Incident Response & Reporting","content":"Report security or operational incidents immediately via the incident response channel. Follow incident response procedures and cooperate with investigations."}
{"id":"rule_090","topic":"Business Continuity & Disaster Recovery","content":"Follow business continuity plans during disruptions. Ensure critical systems and data are recoverable according to defined RTOs and RPOs."}
{"id":"rule_091","topic":"Insurance & Liability","content":"Maintain required insurance coverages. Notify Legal and Finance of incidents that may give rise to claims or liabilities."}
{"id":"rule_092","topic":"Fraud Prevention","content":"Report suspected fraud or misuse of assets. Cooperate with investigations and do not attempt to conceal suspicious activity."}
{"id":"rule_093","topic":"Ethics & Code of Conduct","content":"Adhere to the company Code of Conduct, acting with integrity, honesty, and professionalism in all business dealings."}
{"id":"rule_094","topic":"Non-Compete & Non-Solicitation","content":"Employees may be subject to non-compete or non-solicitation restrictions per their employment agreement. Review terms before outside employment."}
{"id":"rule_095","topic":"Moonlighting & Outside Employment","content":"Employees must disclose outside employment that could conflict with company responsibilities or compete with the business."}
{"id":"rule_096","topic":"Policy Updates & Amendments","content":"Company policies may be updated periodically. Employees will be notified of changes and must acknowledge updated policies as required."}
{"id":"rule_097","topic":"Policy Acknowledgment","content":"Employees must read and acknowledge core policies during onboarding and after major updates. Failure to acknowledge may affect system access."}
{"id":"rule_098","topic":"Disciplinary Procedures","content":"Violations of policies may result in progressive discipline up to termination, depending on severity and frequency of conduct."}
{"id":"rule_099","topic":"Appeals & Grievances","content":"Employees may appeal disciplinary decisions or lodge grievances through established HR procedures; investigations are handled impartially."}
{"id":"rule_100","topic":"Record of Changes & Archive","content":"A master copy of company policies is maintained by HR. Historical versions are archived and accessible for regulatory or audit purposes."}
"""
with open(DATA_PATH, "w", encoding="utf-8") as f:
    f.write(jsonl_text)

print("Saved:", DATA_PATH)
!wc -l "$DATA_PATH"


Saved: /content/drive/MyDrive/company_rag/company_regulations.jsonl
100 /content/drive/MyDrive/company_rag/company_regulations.jsonl


## Step 5 : Embedding and create the data vector store

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import json
from tqdm import tqdm
import chromadb
from openai import OpenAI
import os

# === PATHS ===
BASE_DIR = "/content/drive/MyDrive/company_rag"
CHROMA_DIR = BASE_DIR  # ✅ Parent directory, not the inner folder
DATA_PATH = "/content/drive/MyDrive/company_rag/company_regulations.jsonl"
COLLECTION = "company_regulations"
EMBED_MODEL = "text-embedding-3-large"

# Ensure folder exists
os.makedirs(BASE_DIR, exist_ok=True)

# === Setup Chroma and OpenAI ===
client = OpenAI()
chroma = chromadb.PersistentClient(path=CHROMA_DIR)

collection = chroma.get_or_create_collection(
    name=COLLECTION,
    metadata={"source": "internal_policy_docs"}
)

# === Load and embed dataset ===
with open(DATA_PATH, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

texts = [item["content"] for item in data]
metas = [{"topic": item["topic"], "id": item["id"]} for item in data]

print(f"Embedding {len(texts)} regulations...")

embs = [e.embedding for e in client.embeddings.create(model=EMBED_MODEL, input=texts).data]

# === Add to Chroma ===
for i, emb in enumerate(tqdm(embs)):
    collection.add(
        ids=[data[i]["id"]],
        documents=[texts[i]],
        metadatas=[metas[i]],
        embeddings=[emb]
    )

print(f"✅ Indexed {len(texts)} regulations into {CHROMA_DIR}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Embedding 100 regulations...


100%|██████████| 100/100 [00:07<00:00, 14.22it/s]

✅ Indexed 100 regulations into /content/drive/MyDrive/company_rag


## **Step 6 : Setup the Retriever generator and connecting to the database (we also set our prompt here)**

In [ ]:
# --- 🔧 Setup Cell (run only once) ---
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

CHROMA_DIR = "/content/drive/MyDrive/company_rag"

# Load vector DB
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    persist_directory=CHROMA_DIR,
    collection_name="company_regulations",
    embedding_function=embeddings
)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Load LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create Prompt
template = """
You are an HR assistant helping employees understand company regulations.
Always answer clearly using information from the retrieved context.

If the answer is not in the provided context, say:
"I’m not sure about this policy. Please check with the HR department."

Context:
{context}

Question:
{question}

Answer:
"""
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

# Create the chain (reusable)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

print("✅ RAG system initialized — ready for questions!")


✅ RAG system initialized — ready for questions!


## **Step 7 : Ready to test! Then we can trace and check the retrieval and genewration in LangChain platform)**

In [ ]:
# --- 💬 Question Cell (re-run freely) ---
query = "What benefits are provided to employees after completing the probation period?"
result = rag_chain.invoke({"query": query})

print("🧠 AI Answer:\n", result["result"])


🧠 AI Answer:
 I’m not sure about this policy. Please check with the HR department.


**Examples!**

What is the company’s policy on remote work or working from home?

How many days of annual leave are employees entitled to each year?

Are employees allowed to take unpaid leave, and under what conditions?

What are the company’s rules regarding overtime and compensation for extra hours?

What is the procedure for reporting workplace harassment or discrimination?

Can employees use their personal devices for official work (BYOD policy)?

How is performance evaluation conducted, and how often does it occur?

What is the dress code policy for regular workdays and formal events?

What benefits are provided to employees after completing the probation period?

How should employees request approval for attending external training or conferences?

## **## More Queries!**

In [ ]:
# --- 🧠 Batch Test Cell ---
questions = [
    "What is the company’s policy on remote work or working from home?",
    "How many days of annual leave are employees entitled to each year?",
    "Are employees allowed to take unpaid leave, and under what conditions?",
    "What are the company’s rules regarding overtime and compensation for extra hours?",
    "What is the procedure for reporting workplace harassment or discrimination?",
    "Can employees use their personal devices for official work (BYOD policy)?",
    "How is performance evaluation conducted, and how often does it occur?",
    "What is the dress code policy for regular workdays and formal events?",
    "What benefits are provided to employees after completing the probation period?",
    "How should employees request approval for attending external training or conferences?"
]

results = []

for q in questions:
    res = rag_chain.invoke({"query": q})
    results.append((q, res["result"]))

# --- 🪄 Print all answers nicely ---
for i, (q, a) in enumerate(results, start=1):
    print(f"\n{'='*70}")
    print(f"❓ Question {i}: {q}")
    print(f"🧠 Answer: {a}")



❓ Question 1: What is the company’s policy on remote work or working from home?
🧠 Answer: The company allows remote work only with manager approval. Eligibility for remote work is based on job function, performance, and business needs. Additionally, remote schedules must be agreed upon in writing.

❓ Question 2: How many days of annual leave are employees entitled to each year?
🧠 Answer: I’m not sure about this policy. Please check with the HR department.

❓ Question 3: Are employees allowed to take unpaid leave, and under what conditions?
🧠 Answer: I’m not sure about this policy. Please check with the HR department.

❓ Question 4: What are the company’s rules regarding overtime and compensation for extra hours?
🧠 Answer: Overtime must be approved in advance by a manager. Non-exempt employees will be compensated according to local wage laws for approved overtime hours.

❓ Question 5: What is the procedure for reporting workplace harassment or discrimination?
🧠 Answer: The procedure fo